In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2

In [2]:
baseline = pd.read_csv("../data/processed/baseline_results.csv", parse_dates=["Date"])
sentiment = pd.read_csv("../data/processed/sentiment_results.csv", parse_dates=["Date"])

In [3]:
for df in [baseline, sentiment]:
    df["violation_95"] = (df["y_real"] < df["VaR_95"]).astype(int)
    df["violation_99"] = (df["y_real"] < df["VaR_99"]).astype(int)

In [4]:
def kupiec_test(violations, alpha):
    T = len(violations)
    x = violations.sum()
    p_hat = x / T

    if p_hat == 0 or p_hat == 1:
        return np.nan, np.nan

    LR = -2 * (
        (T - x) * np.log((1 - alpha)/(1 - p_hat)) +
        x * np.log(alpha/p_hat)
    )

    p_value = 1 - chi2.cdf(LR, df=1)

    return LR, p_value

In [5]:
# 95%
LR_b_95, p_b_95 = kupiec_test(baseline["violation_95"], 0.05)
LR_s_95, p_s_95 = kupiec_test(sentiment["violation_95"], 0.05)

# 99%
LR_b_99, p_b_99 = kupiec_test(baseline["violation_99"], 0.01)
LR_s_99, p_s_99 = kupiec_test(sentiment["violation_99"], 0.01)

print("Baseline 95%:  LR =", LR_b_95, "p-value =", p_b_95)
print("Sentiment 95%: LR =", LR_s_95, "p-value =", p_s_95)

print("Baseline 99%:  LR =", LR_b_99, "p-value =", p_b_99)
print("Sentiment 99%: LR =", LR_s_99, "p-value =", p_s_99)

Baseline 95%:  LR = 7.8118834057741395 p-value = 0.005190378179801058
Sentiment 95%: LR = 22.181474705359292 p-value = 2.480557519168869e-06
Baseline 99%:  LR = 0.03971167842775736 p-value = 0.842045371602491
Sentiment 99%: LR = 0.8247187634022 p-value = 0.3638041128228918


In [6]:
#2
def christoffersen_ind_test(violations):
    v = np.asarray(violations, dtype=int)

    # lagged violations
    v0 = v[:-1]
    v1 = v[1:]

    # transition counts
    n00 = np.sum((v0 == 0) & (v1 == 0))
    n01 = np.sum((v0 == 0) & (v1 == 1))
    n10 = np.sum((v0 == 1) & (v1 == 0))
    n11 = np.sum((v0 == 1) & (v1 == 1))

    # transition probabilities
    pi01 = n01 / (n00 + n01) if (n00 + n01) > 0 else 0
    pi11 = n11 / (n10 + n11) if (n10 + n11) > 0 else 0
    pi1  = (n01 + n11) / (n00 + n01 + n10 + n11)

    # avoid log(0)
    eps = 1e-12
    pi01 = np.clip(pi01, eps, 1-eps)
    pi11 = np.clip(pi11, eps, 1-eps)
    pi1  = np.clip(pi1,  eps, 1-eps)

    # log-likelihood under independence
    L0 = (n00 + n10) * np.log(1 - pi1) + (n01 + n11) * np.log(pi1)

    # log-likelihood under Markov alternative
    L1 = (
        n00 * np.log(1 - pi01) +
        n01 * np.log(pi01) +
        n10 * np.log(1 - pi11) +
        n11 * np.log(pi11)
    )

    LR_ind = -2 * (L0 - L1)
    p_value = 1 - chi2.cdf(LR_ind, df=1)

    return LR_ind, p_value

In [7]:
# 95% (alpha = 0.05)
LR_b_95_ind, p_b_95_ind = christoffersen_ind_test(baseline["violation_95"])
LR_s_95_ind, p_s_95_ind = christoffersen_ind_test(sentiment["violation_95"])

# 99% (alpha = 0.01)
LR_b_99_ind, p_b_99_ind = christoffersen_ind_test(baseline["violation_99"])
LR_s_99_ind, p_s_99_ind = christoffersen_ind_test(sentiment["violation_99"])

print("Baseline 95% IND:  LR =", LR_b_95_ind, "p =", p_b_95_ind)
print("Sentiment 95% IND: LR =", LR_s_95_ind, "p =", p_s_95_ind)

print("Baseline 99% IND:  LR =", LR_b_99_ind, "p =", p_b_99_ind)
print("Sentiment 99% IND: LR =", LR_s_99_ind, "p =", p_s_99_ind)

Baseline 95% IND:  LR = 0.0881901203565576 p = 0.7664907762626274
Sentiment 95% IND: LR = 0.04438491881558093 p = 0.8331390999837467
Baseline 99% IND:  LR = 6.680451997050994 p = 0.009747590405772999
Sentiment 99% IND: LR = 1.4257082417011873 p = 0.23246613471076794


In [8]:
#3
from scipy.stats import norm

# Keep only common dates
common_dates = baseline["Date"].isin(sentiment["Date"])
baseline = baseline[common_dates].copy()

sentiment = sentiment[sentiment["Date"].isin(baseline["Date"])].copy()

# Sort to ensure same order
baseline = baseline.sort_values("Date").reset_index(drop=True)
sentiment = sentiment.sort_values("Date").reset_index(drop=True)

def quantile_loss(y, q, alpha):
    y = np.asarray(y, dtype=float)
    q = np.asarray(q, dtype=float)
    indicator = (y < q).astype(int)
    return (alpha - indicator) * (y - q)

def dm_test(loss_diff):
    """
    loss_diff = L_sentiment - L_baseline
    H0: mean(loss_diff) = 0
    """
    d = np.asarray(loss_diff, dtype=float)
    T = len(d)

    d_bar = d.mean()
    var_d = d.var(ddof=1)

    if var_d == 0:
        return np.nan, np.nan, d_bar

    DM_stat = d_bar / np.sqrt(var_d / T)
    p_value = 2 * (1 - norm.cdf(abs(DM_stat)))

    return DM_stat, p_value, d_bar

alpha = 0.05

L_base_95 = quantile_loss(
    baseline["y_real"],
    baseline["VaR_95"],
    alpha
)

L_sent_95 = quantile_loss(
    sentiment["y_real"],
    sentiment["VaR_95"],
    alpha
)

loss_diff_95 = L_sent_95 - L_base_95

DM_95, p_95, mean_diff_95 = dm_test(loss_diff_95)

print("DM test 95%")
print("DM statistic:", DM_95)
print("p-value:", p_95)
print("Mean loss difference (sent - base):", mean_diff_95)

alpha = 0.01

L_base_99 = quantile_loss(
    baseline["y_real"],
    baseline["VaR_99"],
    alpha
)

L_sent_99 = quantile_loss(
    sentiment["y_real"],
    sentiment["VaR_99"],
    alpha
)

loss_diff_99 = L_sent_99 - L_base_99

DM_99, p_99, mean_diff_99 = dm_test(loss_diff_99)

print("DM test 99%")
print("DM statistic:", DM_99)
print("p-value:", p_99)
print("Mean loss difference (sent - base):", mean_diff_99)

DM test 95%
DM statistic: 1.3265121150423138
p-value: 0.18467012491197066
Mean loss difference (sent - base): 4.360323912979745e-05
DM test 99%
DM statistic: 1.21192909306439
p-value: 0.2255395244128371
Mean loss difference (sent - base): 4.0096167127662706e-05


Eval: ES

In [9]:
# ES empirical evaluation function (non-Acerbi–Szekely)
def evaluate_es_empirical(df: pd.DataFrame, alpha: float, y_col: str = "y_real"):
    """
    Empirical ES evaluation:
    1) VaR violations: y_real <= VaR_alpha
    2) Realized tail mean: mean(y_real | violation)
    3) Compare predicted ES with realized tail mean
    4) ES Bias / MAE / RMSE (on violation days)
    """
    level = int(alpha * 100)
    var_col = f"VaR_{level}"
    es_col = f"ES_{level}"

    required = [y_col, var_col, es_col]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns for {level}%: {missing}")

    x = df[required].dropna().copy()
    n = len(x)

    # 1) Identify VaR violations
    violations = x[y_col] <= x[var_col]
    n_viol = int(violations.sum())
    viol_rate = n_viol / n if n > 0 else np.nan

    print(f"\n--- ES Evaluation @ {level}% ---")
    print(f"Observations: {n}")
    print(f"VaR violations: {n_viol} ({viol_rate:.4%})")

    if n_viol == 0:
        print("No VaR violations -> ES metrics cannot be computed.")
        return {
            "alpha": alpha,
            "n_obs": n,
            "n_viol": n_viol,
            "viol_rate": viol_rate,
            "realized_tail_mean": np.nan,
            "pred_es_tail_mean": np.nan,
            "tail_mean_gap": np.nan,
            "es_bias": np.nan,
            "es_mae": np.nan,
            "es_rmse": np.nan,
        }

    tail = x.loc[violations, [y_col, es_col]].copy()

    # 2) Realized tail mean
    realized_tail_mean = tail[y_col].mean()

    # 3) Compare predicted ES with realized tail mean
    pred_es_tail_mean = tail[es_col].mean()
    tail_mean_gap = pred_es_tail_mean - realized_tail_mean

    # 4) ES metrics on violation days
    errors = tail[es_col] - tail[y_col]
    es_bias = errors.mean()
    es_mae = errors.abs().mean()
    es_rmse = np.sqrt((errors ** 2).mean())

    print(f"Realized tail mean (y_real | violation): {realized_tail_mean:.6f}")
    print(f"Predicted ES mean on violations:         {pred_es_tail_mean:.6f}")
    print(f"Tail-mean gap (ES_pred - realized):      {tail_mean_gap:.6f}")
    print(f"ES Bias  (mean(ES_pred - y_real)):       {es_bias:.6f}")
    print(f"ES MAE   (mean(|ES_pred - y_real|)):     {es_mae:.6f}")
    print(f"ES RMSE  (sqrt(mean((ES_pred - y_real)^2))): {es_rmse:.6f}")

    # 6) Interpretation output
    print("Interpretation:")
    if tail_mean_gap > 0:
        print("- ES is less negative than realized tail mean -> tail risk is underestimated.")
    elif tail_mean_gap < 0:
        print("- ES is more negative than realized tail mean -> tail risk estimate is conservative.")
    else:
        print("- ES mean matches realized tail mean exactly.")

    return {
        "alpha": alpha,
        "n_obs": n,
        "n_viol": n_viol,
        "viol_rate": viol_rate,
        "realized_tail_mean": realized_tail_mean,
        "pred_es_tail_mean": pred_es_tail_mean,
        "tail_mean_gap": tail_mean_gap,
        "es_bias": es_bias,
        "es_mae": es_mae,
        "es_rmse": es_rmse,
    }


In [10]:
#Load baseline results and evaluate ES at 95% and 99%
baseline_path = "../data/processed/baseline_results.csv"
baseline_df = pd.read_csv(baseline_path)

if "Date" in baseline_df.columns:
    baseline_df["Date"] = pd.to_datetime(baseline_df["Date"], errors="coerce")
    baseline_df = baseline_df.sort_values("Date")

print("=== Baseline ===")
baseline_95 = evaluate_es_empirical(baseline_df, alpha=0.95, y_col="y_real")
baseline_99 = evaluate_es_empirical(baseline_df, alpha=0.99, y_col="y_real")


=== Baseline ===

--- ES Evaluation @ 95% ---
Observations: 1522
VaR violations: 101 (6.6360%)
Realized tail mean (y_real | violation): -0.026262
Predicted ES mean on violations:         -0.027861
Tail-mean gap (ES_pred - realized):      -0.001598
ES Bias  (mean(ES_pred - y_real)):       -0.001598
ES MAE   (mean(|ES_pred - y_real|)):     0.008643
ES RMSE  (sqrt(mean((ES_pred - y_real)^2))): 0.016601
Interpretation:
- ES is more negative than realized tail mean -> tail risk estimate is conservative.

--- ES Evaluation @ 99% ---
Observations: 1522
VaR violations: 16 (1.0512%)
Realized tail mean (y_real | violation): -0.050760
Predicted ES mean on violations:         -0.058280
Tail-mean gap (ES_pred - realized):      -0.007519
ES Bias  (mean(ES_pred - y_real)):       -0.007519
ES MAE   (mean(|ES_pred - y_real|)):     0.012971
ES RMSE  (sqrt(mean((ES_pred - y_real)^2))): 0.016545
Interpretation:
- ES is more negative than realized tail mean -> tail risk estimate is conservative.


In [11]:
#Load sentiment results and evaluate ES at 95% and 99%
sentiment_path = "../data/processed/sentiment_results.csv"
sentiment_df = pd.read_csv(sentiment_path)

if "Date" in sentiment_df.columns:
    sentiment_df["Date"] = pd.to_datetime(sentiment_df["Date"], errors="coerce")
    sentiment_df = sentiment_df.sort_values("Date")

print("=== Sentiment ===")
sentiment_95 = evaluate_es_empirical(sentiment_df, alpha=0.95, y_col="y_real")
sentiment_99 = evaluate_es_empirical(sentiment_df, alpha=0.99, y_col="y_real")


=== Sentiment ===

--- ES Evaluation @ 95% ---
Observations: 1533
VaR violations: 120 (7.8278%)
Realized tail mean (y_real | violation): -0.024570
Predicted ES mean on violations:         -0.025258
Tail-mean gap (ES_pred - realized):      -0.000689
ES Bias  (mean(ES_pred - y_real)):       -0.000689
ES MAE   (mean(|ES_pred - y_real|)):     0.008222
ES RMSE  (sqrt(mean((ES_pred - y_real)^2))): 0.014203
Interpretation:
- ES is more negative than realized tail mean -> tail risk estimate is conservative.

--- ES Evaluation @ 99% ---
Observations: 1533
VaR violations: 19 (1.2394%)
Realized tail mean (y_real | violation): -0.046428
Predicted ES mean on violations:         -0.045471
Tail-mean gap (ES_pred - realized):      0.000957
ES Bias  (mean(ES_pred - y_real)):       0.000957
ES MAE   (mean(|ES_pred - y_real|)):     0.011725
ES RMSE  (sqrt(mean((ES_pred - y_real)^2))): 0.017223
Interpretation:
- ES is less negative than realized tail mean -> tail risk is underestimated.


In [12]:
# Cell 5 (optional): Compact comparison table
comparison = pd.DataFrame([
    {"Model": "Baseline", "Level": "95%", **{k: v for k, v in baseline_95.items() if k not in ["alpha"]}},
    {"Model": "Baseline", "Level": "99%", **{k: v for k, v in baseline_99.items() if k not in ["alpha"]}},
    {"Model": "Sentiment", "Level": "95%", **{k: v for k, v in sentiment_95.items() if k not in ["alpha"]}},
    {"Model": "Sentiment", "Level": "99%", **{k: v for k, v in sentiment_99.items() if k not in ["alpha"]}},
])

comparison[[
    "Model", "Level", "n_obs", "n_viol", "viol_rate",
    "realized_tail_mean", "pred_es_tail_mean", "tail_mean_gap",
    "es_bias", "es_mae", "es_rmse"
]]


,Model,Level,n_obs,n_viol,viol_rate,realized_tail_mean,pred_es_tail_mean,tail_mean_gap,es_bias,es_mae,es_rmse
0,Baseline,95%,1522,101,0.066360,-0.026262,-0.027861,-0.001598,-0.001598,0.008643,0.016601
1,Baseline,99%,1522,16,0.010512,-0.050760,-0.058280,-0.007519,-0.007519,0.012971,0.016545
2,Sentiment,95%,1533,120,0.078278,-0.024570,-0.025258,-0.000689,-0.000689,0.008222,0.014203
3,Sentiment,99%,1533,19,0.012394,-0.046428,-0.045471,0.000957,0.000957,0.011725,0.017223


In [ ]:
# SAVE ARTIFACTS FOR TABLES/PLOTS (no recomputation in reporting notebook)
from pathlib import Path

OUT_DIR = Path("../data/processed/evaluation")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Use full result files for table stats to avoid accidental mutation from earlier cells.
base_full = pd.read_csv("../data/processed/baseline_results.csv", parse_dates=["Date"])
sent_full = pd.read_csv("../data/processed/sentiment_results.csv", parse_dates=["Date"])

for _df in [base_full, sent_full]:
    _df["violation_95"] = (_df["y_real"] < _df["VaR_95"]).astype(int)
    _df["violation_99"] = (_df["y_real"] < _df["VaR_99"]).astype(int)

# ---------------------------
# Table 1: VaR Backtesting Summary
# ---------------------------
var_rows = []
for model_name, _df in [("Baseline", base_full), ("Sentiment", sent_full)]:
    for level, alpha_tail, vcol in [("95%", 0.05, "violation_95"), ("99%", 0.01, "violation_99")]:
        n_obs = int(len(_df))
        n_viol = int(_df[vcol].sum())
        var_rows.append({
            "Model": model_name,
            "Confidence": level,
            "Observations": n_obs,
            "Violations": n_viol,
            "Violation rate": n_viol / n_obs if n_obs else np.nan,
            "Expected rate": alpha_tail,
        })

var_backtesting_summary = pd.DataFrame(var_rows)
var_backtesting_summary.to_csv(OUT_DIR / "var_backtesting_summary.csv", index=False)

# ---------------------------
# Kupiec / Christoffersen summaries
# ---------------------------
kupiec_summary = pd.DataFrame([
    {"Model": "Baseline",  "Confidence": "95%", "LR_uc": LR_b_95, "p_value": p_b_95},
    {"Model": "Sentiment", "Confidence": "95%", "LR_uc": LR_s_95, "p_value": p_s_95},
    {"Model": "Baseline",  "Confidence": "99%", "LR_uc": LR_b_99, "p_value": p_b_99},
    {"Model": "Sentiment", "Confidence": "99%", "LR_uc": LR_s_99, "p_value": p_s_99},
])
kupiec_summary.to_csv(OUT_DIR / "kupiec_summary.csv", index=False)

christoffersen_summary = pd.DataFrame([
    {"Model": "Baseline",  "Confidence": "95%", "LR_ind": LR_b_95_ind, "p_value": p_b_95_ind},
    {"Model": "Sentiment", "Confidence": "95%", "LR_ind": LR_s_95_ind, "p_value": p_s_95_ind},
    {"Model": "Baseline",  "Confidence": "99%", "LR_ind": LR_b_99_ind, "p_value": p_b_99_ind},
    {"Model": "Sentiment", "Confidence": "99%", "LR_ind": LR_s_99_ind, "p_value": p_s_99_ind},
])
christoffersen_summary.to_csv(OUT_DIR / "christoffersen_summary.csv", index=False)

# ---------------------------
# Table 2: Empirical ES Evaluation Summary
# ---------------------------
es_empirical_summary = pd.DataFrame([
    {"Model": "Baseline",  "Confidence": "95%", **{k: v for k, v in baseline_95.items() if k != "alpha"}},
    {"Model": "Baseline",  "Confidence": "99%", **{k: v for k, v in baseline_99.items() if k != "alpha"}},
    {"Model": "Sentiment", "Confidence": "95%", **{k: v for k, v in sentiment_95.items() if k != "alpha"}},
    {"Model": "Sentiment", "Confidence": "99%", **{k: v for k, v in sentiment_99.items() if k != "alpha"}},
])

# Rename to thesis-friendly labels
es_empirical_summary = es_empirical_summary.rename(columns={
    "n_obs": "Observations",
    "n_viol": "Violations",
    "viol_rate": "Violation rate",
    "realized_tail_mean": "Realized tail mean",
    "pred_es_tail_mean": "Predicted ES mean (violations)",
    "tail_mean_gap": "Tail mean gap (ES_pred - realized)",
    "es_bias": "ES Bias",
    "es_mae": "ES MAE",
    "es_rmse": "ES RMSE",
})
es_empirical_summary.to_csv(OUT_DIR / "es_empirical_summary.csv", index=False)

# ---------------------------
# DM test summary
# ---------------------------
dm_summary = pd.DataFrame([
    {"Confidence": "95%", "DM statistic": DM_95, "p_value": p_95, "mean_loss_diff_sent_minus_base": mean_diff_95},
    {"Confidence": "99%", "DM statistic": DM_99, "p_value": p_99, "mean_loss_diff_sent_minus_base": mean_diff_99},
])
dm_summary.to_csv(OUT_DIR / "dm_test_summary.csv", index=False)

# ---------------------------
# Plot-ready source (95%) so plotting notebook doesn't recompute joins/violations
# ---------------------------
plot_95 = (
    base_full[["Date", "y_real", "VaR_95", "ES_95", "violation_95"]]
    .rename(columns={
        "y_real": "y_real_baseline",
        "VaR_95": "VaR_95_baseline",
        "ES_95": "ES_95_baseline",
        "violation_95": "violation_95_baseline",
    })
    .merge(
        sent_full[["Date", "y_real", "VaR_95", "ES_95", "violation_95"]]
        .rename(columns={
            "y_real": "y_real_sentiment",
            "VaR_95": "VaR_95_sentiment",
            "ES_95": "ES_95_sentiment",
            "violation_95": "violation_95_sentiment",
        }),
        on="Date",
        how="inner",
    )
    .sort_values("Date")
)
plot_95.to_csv(OUT_DIR / "plot_source_95.csv", index=False)


# Additional per-model and 99% plot artifacts
base_full.to_csv(OUT_DIR / "baseline_oos_with_violations.csv", index=False)
sent_full.to_csv(OUT_DIR / "sentiment_oos_with_violations.csv", index=False)

base_full.loc[base_full["violation_95"] == 1, ["Date", "y_real", "VaR_95", "ES_95"]].to_csv(OUT_DIR / "baseline_tail_95.csv", index=False)
base_full.loc[base_full["violation_99"] == 1, ["Date", "y_real", "VaR_99", "ES_99"]].to_csv(OUT_DIR / "baseline_tail_99.csv", index=False)

sent_full.loc[sent_full["violation_95"] == 1, ["Date", "y_real", "VaR_95", "ES_95"]].to_csv(OUT_DIR / "sentiment_tail_95.csv", index=False)
sent_full.loc[sent_full["violation_99"] == 1, ["Date", "y_real", "VaR_99", "ES_99"]].to_csv(OUT_DIR / "sentiment_tail_99.csv", index=False)

plot_99 = (
    base_full[["Date", "y_real", "VaR_99", "ES_99", "violation_99"]]
    .rename(columns={
        "y_real": "y_real_baseline",
        "VaR_99": "VaR_99_baseline",
        "ES_99": "ES_99_baseline",
        "violation_99": "violation_99_baseline",
    })
    .merge(
        sent_full[["Date", "y_real", "VaR_99", "ES_99", "violation_99"]]
        .rename(columns={
            "y_real": "y_real_sentiment",
            "VaR_99": "VaR_99_sentiment",
            "ES_99": "ES_99_sentiment",
            "violation_99": "violation_99_sentiment",
        }),
        on="Date",
        how="inner",
    )
    .sort_values("Date")
)
plot_99.to_csv(OUT_DIR / "plot_source_99.csv", index=False)

print("- baseline_oos_with_violations.csv")
print("- sentiment_oos_with_violations.csv")
print("- baseline_tail_95.csv")
print("- baseline_tail_99.csv")
print("- sentiment_tail_95.csv")
print("- sentiment_tail_99.csv")
print("- plot_source_99.csv")

print("Saved evaluation artifacts to:", OUT_DIR.resolve())
print("- var_backtesting_summary.csv")
print("- es_empirical_summary.csv")
print("- kupiec_summary.csv")
print("- christoffersen_summary.csv")
print("- dm_test_summary.csv")
print("- plot_source_95.csv")

var_backtesting_summary, es_empirical_summary


